In [1]:
import os
import time
import shutil
import librosa
import numpy as np
import tensorflow as tf
import cPickle as pickle
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

### Audio data

In [2]:
pickle_file = "../../my_features/US8K_logmel128_patchslice.pickle"
with open(pickle_file, "rb") as f:
    dataset = pickle.load(f)
    X_train_all = dataset["X_train"]/10
    Y_train_all = dataset["Y_train"]
    X_valid_all = dataset["X_valid"]/10
    Y_valid_all = dataset["Y_valid"]
    X_test1_all = dataset["X_test1"]/10
    Y_test1_all = dataset["Y_test1"]
    X_test2_all = dataset["X_test2"]/10
    Y_test2_all = dataset["Y_test2"]
    del dataset

In [3]:
print np.sum(Y_train_all, axis = 0)
print np.sum(Y_train_all)
print np.unique(np.argmax(Y_train_all, axis=1), return_counts=True)

[ 2793.   765.  2763.  2027.  2321.  2704.   213.  2625.  2717.  2800.]
21728.0
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([2793,  765, 2763, 2027, 2321, 2704,  213, 2625, 2717, 2800]))


In [4]:
print ('training data: ' , X_train_all.shape, Y_train_all.shape)

print (np.sum(Y_train_all, axis=0))

('training data: ', (21728, 16384), (21728, 10))
[ 2793.   765.  2763.  2027.  2321.  2704.   213.  2625.  2717.  2800.]


In [5]:
# ## Use 2 classes as training data
# X_train = X_train_all[(np.argmax(Y_train_all, axis=1) == 4)+(np.argmax(Y_train_all, axis=1) == 0)]
# Y_train = Y_train_all[(np.argmax(Y_train_all, axis=1) == 4)+(np.argmax(Y_train_all, axis=1) == 0)][:,(0,4)]

# X_valid = X_valid_all[(np.argmax(Y_valid_all, axis=1) == 4)+(np.argmax(Y_valid_all, axis=1) == 0)]
# Y_valid = Y_valid_all[(np.argmax(Y_valid_all, axis=1) == 4)+(np.argmax(Y_valid_all, axis=1) == 0)][:,(0,4)]

# X_test1 = X_test1_all[(np.argmax(Y_test1_all, axis=1) == 4)+(np.argmax(Y_test1_all, axis=1) == 0)]
# Y_test1 = Y_test1_all[(np.argmax(Y_test1_all, axis=1) == 4)+(np.argmax(Y_test1_all, axis=1) == 0)][:,(0,4)]

# X_test2 = X_test2_all[(np.argmax(Y_test2_all, axis=1) == 4)+(np.argmax(Y_test2_all, axis=1) == 0)]
# Y_test2 = Y_test2_all[(np.argmax(Y_test2_all, axis=1) == 4)+(np.argmax(Y_test2_all, axis=1) == 0)][:,(0,4)]

In [6]:

## Use 5 classes as training data
# X_train = X_train_all[(np.argmax(Y_train_all, axis=1) == 4)+(np.argmax(Y_train_all, axis=1) == 3)+
#                       (np.argmax(Y_train_all, axis=1) == 2)+(np.argmax(Y_train_all, axis=1) == 8)+
#                       (np.argmax(Y_train_all, axis=1) == 7)+(np.argmax(Y_train_all, axis=1) == 5)]
# Y_train = Y_train_all[(np.argmax(Y_train_all, axis=1) == 4)+(np.argmax(Y_train_all, axis=1) == 3)+
#                       (np.argmax(Y_train_all, axis=1) == 2)+(np.argmax(Y_train_all, axis=1) == 8)+
#                       (np.argmax(Y_train_all, axis=1) == 7)+(np.argmax(Y_train_all, axis=1) == 5)][:,(2,3,4,5,8,7)]

# X_valid = X_valid_all[(np.argmax(Y_valid_all, axis=1) == 4)+(np.argmax(Y_valid_all, axis=1) == 3)+
#                       (np.argmax(Y_valid_all, axis=1) == 2)+(np.argmax(Y_valid_all, axis=1) == 8)+ 
#                       (np.argmax(Y_valid_all, axis=1) == 7)+(np.argmax(Y_valid_all, axis=1) == 5)]
# Y_valid = Y_valid_all[(np.argmax(Y_valid_all, axis=1) == 4)+(np.argmax(Y_valid_all, axis=1) == 3)+
#                       (np.argmax(Y_valid_all, axis=1) == 2)+(np.argmax(Y_valid_all, axis=1) == 8)+ 
#                       (np.argmax(Y_valid_all, axis=1) == 7)+(np.argmax(Y_valid_all, axis=1) == 5)][:,(2,3,4,5,8,7)]

# X_test1 = X_test1_all[(np.argmax(Y_test1_all, axis=1) == 4)+(np.argmax(Y_test1_all, axis=1) == 3)+
#                       (np.argmax(Y_test1_all, axis=1) == 2)+(np.argmax(Y_test1_all, axis=1) == 8)+
#                       (np.argmax(Y_test1_all, axis=1) == 7)+(np.argmax(Y_test1_all, axis=1) == 5)]
# Y_test1 = Y_test1_all[(np.argmax(Y_test1_all, axis=1) == 4)+(np.argmax(Y_test1_all, axis=1) == 3)+
#                       (np.argmax(Y_test1_all, axis=1) == 2)+(np.argmax(Y_test1_all, axis=1) == 8)+
#                       (np.argmax(Y_test1_all, axis=1) == 7)+(np.argmax(Y_test1_all, axis=1) == 5)][:,(2,3,4,5,8,7)]

# X_test2 = X_test2_all[(np.argmax(Y_test2_all, axis=1) == 4)+(np.argmax(Y_test2_all, axis=1) == 3)+
#                       (np.argmax(Y_test2_all, axis=1) == 2)+(np.argmax(Y_test2_all, axis=1) == 8)+
#                       (np.argmax(Y_test2_all, axis=1) == 7)+(np.argmax(Y_test2_all, axis=1) == 5)]
# Y_test2 = Y_test2_all[(np.argmax(Y_test2_all, axis=1) == 4)+(np.argmax(Y_test2_all, axis=1) == 3)+
#                       (np.argmax(Y_test2_all, axis=1) == 2)+(np.argmax(Y_test2_all, axis=1) == 8)+
#                       (np.argmax(Y_test2_all, axis=1) == 7)+(np.argmax(Y_test2_all, axis=1) == 5)][:,(2,3,4,5,8,7)]


## Use the whole data set as training data

X_train = X_train_all
Y_train = Y_train_all

X_valid = X_valid_all
Y_valid = Y_valid_all

X_test1 = X_test1_all
Y_test1 = Y_test1_all

X_test2 = X_test2_all
Y_test2 = Y_test2_all


In [7]:
IMG_SIZE = 128
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_FLAT_SIZE = IMG_WIDTH*IMG_HEIGHT
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT)
N_LABELS = Y_train.shape[1]

In [8]:
print (Y_train.shape)
print (np.unique(np.argmax(Y_train, axis=1), return_counts=True))

(21728, 10)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([2793,  765, 2763, 2027, 2321, 2704,  213, 2625, 2717, 2800]))


In [9]:
print (X_train.shape)

(21728, 16384)


## 1. Convolutional network

In [10]:
## Helper function

## Weight initialization
def new_weights(shape, stddev):
    ## Xavier intialization
    initial = tf.truncated_normal(shape=shape, stddev=stddev,dtype=tf.float32)
    return tf.Variable(initial)
## Biases initialization
def new_biases(length):
    initial = tf.constant(value=0, shape=[length], dtype=tf.float32)
    return tf.Variable(initial)
def get_layer_shape(in_layer):
    shape = in_layer.get_shape()
#     n_features = shape[1:4].num_elements()
    return np.array([shape[1:2].num_elements(), shape[2:3].num_elements(), shape[3:4].num_elements()])

## Fully connected layer
def new_fc_layer(in_layer, n_in_features, n_out_features, relu=True):
    stddev = np.sqrt(np.float(2)/(n_in_features + n_out_features))
    
    weights = new_weights([n_in_features, n_out_features], stddev)
    biases = new_biases(n_out_features)
    out_layer = tf.matmul(in_layer, weights) + biases
    if relu:
        out_layer = tf.nn.relu(out_layer)
    return out_layer, weights
#Helper function creates conv layer
def new_conv_layer(in_layer, filter_height, filter_width, in_channels, n_filter, pooling=True):
    in_layer_shape = get_layer_shape(in_layer)
    in_features = np.prod(in_layer_shape[0:2])
    if(pooling):
        out_features = (in_layer_shape[0] - filter_height + 1)*(in_layer_shape[1] - filter_width + 1)/8
                                    
    else:
        out_features = (in_layer_shape[0] - filter_height + 1)*(in_layer_shape[1] - filter_width + 1)
        
#     print in_features, out_features
    stddev = np.sqrt(np.float(2)/(in_features+out_features))
    weights = new_weights([filter_height, filter_width, in_channels, n_filter], stddev)
    biases = new_biases(n_filter)
    out_layer = tf.nn.conv2d(in_layer, weights, strides=[1,1,1,1], padding='VALID')
    out_layer += biases
    if pooling:
        out_layer = tf.nn.max_pool(out_layer, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    out_layer = tf.nn.relu(out_layer)
    return out_layer, weights
def flatten_layer(in_layer):
    shape = in_layer.get_shape()
    n_features = shape[1:4].num_elements()
    layer_flat = tf.reshape(in_layer, [-1, n_features])
    return layer_flat, n_features

In [11]:
X_cnn = tf.placeholder(tf.float32, [None, IMG_FLAT_SIZE])
X_cnn_train = tf.reshape(X_cnn, [-1, IMG_WIDTH, IMG_HEIGHT, 1])
Y = tf.placeholder(tf.float32, [None, N_LABELS])
keep_prob = tf.placeholder(tf.float32)
print (X_cnn)
print (X_cnn_train)

Tensor("Placeholder:0", shape=(?, 16384), dtype=float32)
Tensor("Reshape:0", shape=(?, 128, 128, 1), dtype=float32)


In [12]:
## 3 convolutional layers
n_conv_layers = 3

filter_heights = dict(zip(range(n_conv_layers+1), [0, 15, 16, 17]))
filter_widths = dict(zip(range(n_conv_layers+1), [0, 2, 3, 4]))
n_filters = dict(zip(range(n_conv_layers+1), [1, 12, 24, 24])) 
pooling = dict(zip(range(n_conv_layers+1), [False, True, True, False]))

## Helper function to create convolultional layers
def make_conv_layer(input_data, n_conv_layers, filter_heights, filter_widths, n_filters, dropout=True):
    conv_layers = [0]*(n_conv_layers+1)
    conv_layers[0] = input_data
    weights = [0]*n_conv_layers
    for f in range(1, n_conv_layers+1):
        
        conv_layers[f], weights[f-1] = new_conv_layer(in_layer=conv_layers[f-1], 
                                                          filter_height=filter_heights[f],
                                                          filter_width=filter_widths[f],
                                                          in_channels=n_filters[f-1], 
                                                          n_filter=n_filters[f], 
                                                          pooling=pooling[f])
    if(dropout):
        drop = tf.nn.dropout(conv_layers[n_conv_layers], keep_prob=keep_prob)
        conv_layers.append(drop)
    return (conv_layers, weights)

conv_layers, conv_weights = make_conv_layer(input_data=X_cnn_train, 
                                            n_conv_layers=n_conv_layers, 
                                            filter_heights=filter_heights,
                                            filter_widths=filter_widths,
                                            n_filters=n_filters, 
                                            dropout=True)

for l, layer in zip(range(n_conv_layers + 1), conv_layers):
    print ("Conv Layer {0}: {1}".format(l, layer))
print ("Conv layer {1} after dropout: {0}".format(conv_layers[-1], n_conv_layers))
## Flatten layer
layer_flat, n_features = flatten_layer(conv_layers[-1])
print ("Flatten layer: {0}, number of features: {1}".format(layer_flat, n_features))

Conv Layer 0: Tensor("Reshape:0", shape=(?, 128, 128, 1), dtype=float32)
Conv Layer 1: Tensor("Relu:0", shape=(?, 57, 63, 12), dtype=float32)
Conv Layer 2: Tensor("Relu_1:0", shape=(?, 21, 30, 24), dtype=float32)
Conv Layer 3: Tensor("Relu_2:0", shape=(?, 5, 27, 24), dtype=float32)
Conv layer 3 after dropout: Tensor("dropout/mul:0", shape=(?, 5, 27, 24), dtype=float32)
Flatten layer: Tensor("Reshape_1:0", shape=(?, 3240), dtype=float32), number of features: 3240


In [13]:
## Fully-connected layer
## Input: flatten layer resulted from conv layers
fc_size = 256
layer_fc1, weights_fc_1 = new_fc_layer(layer_flat, n_in_features=n_features, n_out_features=fc_size)
print ("Fully connected layer 1: {0}".format(layer_fc1))

layer_fc1_drop = tf.nn.dropout(layer_fc1, keep_prob=keep_prob)
print ("Fully connected layer 1 after droppout: {0}".format(layer_fc1_drop))

layer_fc2, weights_fc_2 = new_fc_layer(layer_fc1_drop, n_in_features=fc_size, n_out_features=N_LABELS, relu=False)
print ("Fully connected layer 2: {0}".format(layer_fc2))

# layer_fc2_drop = tf.nn.dropout(layer_fc2, keep_prob=keep_prob)
# print (layer_fc2_drop)

y_nn_softmax = tf.nn.softmax(layer_fc2)
print ("Softmax output: {0}".format(y_nn_softmax))

Fully connected layer 1: Tensor("Relu_3:0", shape=(?, 256), dtype=float32)
Fully connected layer 1 after droppout: Tensor("dropout_1/mul:0", shape=(?, 256), dtype=float32)
Fully connected layer 2: Tensor("add_4:0", shape=(?, 10), dtype=float32)
Softmax output: Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [14]:
LEARNING_RATE = 1e-3
BETA = 1e-3
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=Y)
print (cross_entropy)
# cost = tf.reduce_mean(cross_entropy)
cost = tf.reduce_mean(cross_entropy + BETA*tf.nn.l2_loss(weights_fc_1) +  BETA*tf.nn.l2_loss(weights_fc_2))
print (cost)
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
minimize_cost = optimizer.minimize(cost)

grads = optimizer.compute_gradients(cost,[weights_fc_1, weights_fc_2])
y_true = tf.argmax(Y, dimension=1)
y_pred = tf.argmax(y_nn_softmax, dimension=1)

print grads[0]
print grads[1]

Tensor("Reshape_4:0", shape=(?,), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
(<tf.Tensor 'gradients_1/AddN_1:0' shape=(3240, 256) dtype=float32>, <tensorflow.python.ops.variables.Variable object at 0x7fcbcfcf9710>)
(<tf.Tensor 'gradients_1/AddN:0' shape=(256, 10) dtype=float32>, <tensorflow.python.ops.variables.Variable object at 0x7fcbcfcdcad0>)


In [15]:
# #slice data for testing model
# X_train = X_train[:1000]
# Y_train = Y_train[:1000]

# print (X_train.shape)
# print (Y_train.shape)
# print (np.unique(np.argmax(Y_train, axis=1), return_counts=True))

In [16]:
def plot_conv_weights(session, weights, weights_name, n_channels, epoch):
    n_channels -= 1
    
    for name, weight in zip(weights_name, weights):
        w = session.run(weight)
        w_min = np.min(w)
        w_max = np.max(w)
        n_filters = w.shape[3]
        height, width = w.shape[0:2]
        n_grids = int(np.ceil(np.sqrt(n_filters)))
        fig, axes = plt.subplots(n_grids, n_grids)
        for i, ax in enumerate(axes.flat):
            # Only plot the valid filter-weights.
            if i<n_filters:
                # Get the weights for the i'th filter of the input channel.
                # See new_conv_layer() for details on the format
                # of this 4-dim tensor.
                img = w[:, :, n_channels, i]

                # Plot image.
                ax.imshow(img, 
                          vmin=w_min, vmax=w_max, 
                          cmap='nipy_spectral')
                
                
            # Remove ticks from the plot.
            ax.set_xticks([])
            ax.set_yticks([])
            fig.suptitle(name + ', epoch: ' + str(epoch)+', size: (' + str(height) + ',' + str(width) + ')', 
                         fontsize=10)
            dir_path = './weights_img/'
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
            fig.savefig(dir_path + name + ', epoch: ' + str(epoch) + '.png')
            plt.close(fig)

In [17]:
%matplotlib inline

In [19]:
classID = {
    0: "AirCon",
    1: "Car horn",
    2: "Children playing",
    3: "Dog bark",
    4: "Drilling",
    5: "Engine idling",
    6: "Gun shot",
    7: "Jackhammer",
    8: "Siren",
    9: "Street music"
}

BATCH_SIZE = 50
TRAINING_EPOCHS = 3
np.random.seed(2017)

## Helper function for optimization
def optimize(train_x, train_y, n_epochs, batch_size, session, saver):
        n_samples = train_x.shape[0]
        sample_IDs = np.arange(n_samples)
        np.random.shuffle(sample_IDs)
        train_x_p, train_y_p = train_x[sample_IDs], train_y[sample_IDs]
        n_iterations = np.int(np.floor(n_samples/batch_size))+1
        start_time = time.time()
        cost_history = np.empty(shape=[1],dtype=float)
        print "Training......."
        print "-- Elapsed time -- Epoch -- Cost value -- fc_1 median abs -- fc_2 median abs --"

        for epoch in np.arange(n_epochs+1):
            np.random.shuffle(sample_IDs)
            train_x_p, train_y_p = train_x[sample_IDs], train_y[sample_IDs]
            for itr in np.arange(n_iterations):
                start = (itr * batch_size) % (n_samples - batch_size)
                batch_x, batch_y = train_x_p[start:start + batch_size], train_y_p[start:start + batch_size]
                feed_dict_train = {X_cnn: batch_x, Y: batch_y, keep_prob: 0.5}
                _, c = session.run([minimize_cost, cost], feed_dict=feed_dict_train)
                g = session.run([grad for grad, _ in grads], feed_dict=feed_dict_train)
                
                
            if(epoch % (n_epochs/10) == 0):
                print "-- {:12.6f} -- {:5d} -- {:10.5f} -- {:15.11f} -- {:15.11f} --".format((time.time() - start_time), 
                                                                                    epoch, 
                                                                                    c, 
                                                                                    np.median(np.absolute(g[0])), 
                                                                                    np.median(np.absolute(g[1])))
            cost_history = np.append(cost_history,c)
#             Draw weights of convolutional layer
            if(epoch % (n_epochs/3) == 0):
                plot_conv_weights(session, [conv_weights[0], conv_weights[1], conv_weights[2]], 
                                  ['conv_1', 'conv_2', 'conv_3'], 1, epoch)
                
            
            
            
            
            
#         Save model in folder cnn_model
        dir_path = './cnn_model/'
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        saver.save(sess, 'cnn_model/new_cnn')
        
#         print running time and output cost value graph
        print ("---Running time: %s seconds ---" % (time.time() - start_time))
        print ('*'*50)
        fig = plt.figure(figsize=(10,5))
        plt.plot(cost_history)
        plt.axis([0,epoch,0,np.max(cost_history)])
        plt.show()

def output_log_file(train, valid, test1, test2):
    file_name = pickle_file.split('/')[-1].split('.')[0]
    with open("log/logfile.txt", "ab") as text_file:
        text_file.write('='*60)
        text_file.write('\n')
        text_file.write("Time: {0}\n".format(datetime.now()))
        text_file.write("Data: " + file_name +"\n")    
        
        
#         text_file.write("Number of input samples: {:6d}\n".format(N_SA)
        text_file.write("Number of input features: {:5d}\n".format(IMG_FLAT_SIZE))
#         text_file.write("Number of input labels: {:5d}\n".format(N_LABELS)
        text_file.write("Number of convolutional layer: {0}\n".format(n_conv_layers))
        text_file.write("\tFilter size:\t")
        for idx in np.arange(1, n_conv_layers + 1):
            text_file.write('{0}\t'.format(filter_size[idx]))
        text_file.write("\n")
        text_file.write("\tNumber of filter:\t")
        for idx in np.arange(1, n_conv_layers + 1):
            text_file.write('{0}\t'.format(n_filter[idx]))
        text_file.write("\n")
        text_file.write("Number of fully-connected layer: {0}\n".format(n_fc_layers))
        text_file.write("Hidden units: {:3d}  - {:3d}\n".format(fc_size, N_LABELS))
        text_file.write("Training epochs: {:5d}\n".format(TRAINING_EPOCHS))
        text_file.write("Batch size: {:3d}\n".format(BATCH_SIZE))
                        
        text_file.write('\tTrain\tValid\tTest1\tYoutube\n')
        text_file.write("Fscore\t {:1.2f} \t {:1.2f} \t {:1.2f} \t {:1.2f} \n".format(train[0], 
                                                                                      valid[0], 
                                                                                      test1[0], 
                                                                                      test2[0]))
        text_file.write("Acc\t {:1.2f} \t {:1.2f} \t {:1.2f} \t {:1.2f} \n".format(train[1], 
                                                                                   valid[1], 
                                                                                   test1[1], 
                                                                                   test2[1]))
        
        

## Helper function to print confusion matrix
def make_prediction(test_x, test_y, session, batch_size):
    print "Making prediction......."
    start_time = time.time()
    n_samples = test_x.shape[0]
    sample_IDs = np.arange(n_samples)
    n_iterations = np.int(np.floor(n_samples/batch_size))+1
    pred = np.zeros(n_samples)
    true = np.zeros(n_samples)
    for itr in np.arange(n_iterations):
        start = (itr * batch_size) % (n_samples - batch_size)
        batch_x, batch_y = test_x[start:start + batch_size], test_y[start:start + batch_size]
        feed_dict_test = {X_cnn: batch_x, Y: batch_y, keep_prob: 1}
        pred[start:start + batch_size], true[start:start + batch_size] = session.run([y_pred, y_true], feed_dict=feed_dict_test)

## Performance Evaluation metrics
    ## Accuracy
    accuracy = accuracy_score(true, pred)
    print ("Accuracy: {:3.2f}".format(accuracy))
    ## F-score
    if(N_LABELS == 2):
        p,r,f,s = precision_recall_fscore_support(true, pred, average='binary')
    else:
        p,r,f,s = precision_recall_fscore_support(true, pred, average='macro')
    print ("F-Score: {:3.2f}".format(f))
    
    ## Confusion matrix
    print ("Confusion Matrix")
    print confusion_matrix(true, pred)
#     print ("Analyzing result")
#     for ID, name in classID.items():
#         x = pred[(true == ID)*(pred != ID)]  
#         unique_class, unique_count = np.unique(x, return_counts=True)
#         print ('-'*50)
#         print " Class {0} is mistaken with: ".format(name)
#         for c, y in zip(unique_class, unique_count):        
#             print "# {0}: {1} samples".format(classID[c], y)
#     print ('-'*50)
    print ("---Running time: {0} seconds ---".format((time.time() - start_time)))
    print ('*'*50)
    return accuracy, f
                    

In [ ]:


with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    optimize(X_train, Y_train, TRAINING_EPOCHS, BATCH_SIZE, sess, saver)
    
    print ("Making prediction on training set")
    train = make_prediction(X_train, Y_train, sess, BATCH_SIZE)
    print ("Making prediction on validation set")
    valid = make_prediction(X_valid, Y_valid, sess, BATCH_SIZE)
    print ("Making training prediction on test 1 set")
    test1 = make_prediction(X_test1, Y_test1, sess, BATCH_SIZE)
    print ("Making training prediction on test 2 set")
    test2 = make_prediction(X_test2, Y_test2, sess, BATCH_SIZE)
#     output_log_file(train, valid, test1, test2)

    
    
    

Training.......
-- Elapsed time -- Epoch -- Cost value -- fc_1 median abs -- fc_2 median abs --
